In [1]:
import numpy as np
import pandas as pd
from tensorflow import keras

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [5]:
fashion_mnist = keras.datasets.fashion_mnist
(train_images, train_label), (test_images, test_label) = fashion_mnist.load_data()

In [6]:
print(len(train_images),len(train_label))
print(len(test_images),len(test_label))

60000 60000
10000 10000


In [7]:
train_images.shape

(60000, 28, 28)

In [8]:
from keras.utils import np_utils

In [10]:
pt_x_train = []
pt_y_train = []
pt_x_test = []
pt_y_test = []

tl_x_train = []
tl_y_train = []
tl_x_test = []
tl_y_test = []

m=60000

for i in range(m):
    if train_label[i] < 5:
        pt_x_train.append(train_images[i] / 255)
        pt_y_train.append(train_label[i])
    else:
        tl_x_train.append(train_images[i] / 255)
        tl_y_train.append(train_label[i])

m2 = 10000

for i in range(m2):
    if test_label[i] < 5:
        pt_x_test.append(test_images[i] / 255)
        pt_y_test.append(test_label[i])
    else:
        tl_x_test.append(test_images[i] / 255)
        tl_y_test.append(test_label[i])
                         
pt_x_train = np.asarray(pt_x_train).reshape(-1,28,28,1)
pt_x_test = np.asarray(pt_x_test).reshape(-1,28,28,1)
pt_y_train = np_utils.to_categorical(np.asarray(pt_y_train))
pt_y_test = np_utils.to_categorical(np.asarray(pt_y_test))

tl_x_train = np.asarray(tl_x_train).reshape(-1,28,28,1)
tl_x_test = np.asarray(tl_x_test).reshape(-1,28,28,1)
tl_y_train = np_utils.to_categorical(np.asarray(tl_y_train))
tl_y_test = np_utils.to_categorical(np.asarray(tl_y_test))

                         
print(pt_x_train.shape,pt_y_train.shape)
print(pt_x_test.shape,pt_y_test.shape)

print(tl_x_train.shape,tl_y_train.shape)
print(tl_x_test.shape,tl_y_test.shape)    

(30000, 28, 28, 1) (30000, 5)
(5000, 28, 28, 1) (5000, 5)
(30000, 28, 28, 1) (30000, 10)
(5000, 28, 28, 1) (5000, 10)


In [11]:
from keras.models import Sequential,Model
from keras.layers import Conv2D,Dense,Activation,MaxPool2D,Dropout,Flatten

In [12]:
model = Sequential()

# (28,28,1)

model.add(Conv2D(32,(5,5),input_shape=(28,28,1),activation='relu'))
# (24,24,32)

model.add(Conv2D(16,(5,5),activation='relu'))
# (20,20,16)

model.add(MaxPool2D(pool_size=(2,2)))
# (10,10,16)

model.add(Conv2D(8,(3,3),activation='relu'))
# (8,8,8)

model.add(Flatten())

model.add(Dropout(0.4))

model.add(Dense(128,activation='relu'))

model.add(Dense(64,activation='relu'))

model.add(Dense(5,activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 24, 24, 32)        832       
                                                                 
 conv2d_1 (Conv2D)           (None, 20, 20, 16)        12816     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 10, 10, 16)       0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 8, 8, 8)           1160      
                                                                 
 flatten (Flatten)           (None, 512)               0         
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                        

In [13]:
model.compile(optimizer='adam',
             loss='categorical_crossentropy',
             metrics=['accuracy'])

In [14]:
model.fit(pt_x_train,pt_y_train,
         validation_data=(pt_x_test,pt_y_test),
         epochs=10,
         batch_size=100,
         verbose=2,
         shuffle=True)

Epoch 1/10
300/300 - 15s - loss: 0.5361 - accuracy: 0.7893 - val_loss: 0.3535 - val_accuracy: 0.8714 - 15s/epoch - 51ms/step
Epoch 2/10
300/300 - 16s - loss: 0.3377 - accuracy: 0.8748 - val_loss: 0.2849 - val_accuracy: 0.9020 - 16s/epoch - 53ms/step
Epoch 3/10
300/300 - 15s - loss: 0.2830 - accuracy: 0.8960 - val_loss: 0.2534 - val_accuracy: 0.9092 - 15s/epoch - 49ms/step
Epoch 4/10
300/300 - 13s - loss: 0.2574 - accuracy: 0.9062 - val_loss: 0.2326 - val_accuracy: 0.9182 - 13s/epoch - 44ms/step
Epoch 5/10
300/300 - 13s - loss: 0.2388 - accuracy: 0.9130 - val_loss: 0.2289 - val_accuracy: 0.9166 - 13s/epoch - 44ms/step
Epoch 6/10
300/300 - 13s - loss: 0.2244 - accuracy: 0.9179 - val_loss: 0.2126 - val_accuracy: 0.9222 - 13s/epoch - 44ms/step
Epoch 7/10
300/300 - 13s - loss: 0.2134 - accuracy: 0.9220 - val_loss: 0.2050 - val_accuracy: 0.9266 - 13s/epoch - 44ms/step
Epoch 8/10
300/300 - 13s - loss: 0.2050 - accuracy: 0.9257 - val_loss: 0.2114 - val_accuracy: 0.9234 - 13s/epoch - 44ms/step


In [15]:
for layer in model.layers[:5]:
    layer.trainable = False

In [16]:
x = model.layers[4].output
x = Dropout(0.5)(x)
x = Dense(32,activation='relu')(x)
x = Dense(16,activation='relu')(x)
predictions = Dense(10,activation='softmax')(x)

In [17]:
tl_model = Model(model.input,predictions)

In [ ]:
tl_model.summary()

In [19]:
tl_model.compile(optimizer='adam',
          loss='categorical_crossentropy',
          metrics=['accuracy'])

In [21]:
tl_model.fit(tl_x_train,tl_y_train,
            validation_data=(tl_x_test,tl_y_test),
            batch_size=100,
            epochs=10,
            verbose=2,
            shuffle=True)

Epoch 1/10
300/300 - 3s - loss: 0.1540 - accuracy: 0.9453 - val_loss: 0.1276 - val_accuracy: 0.9576 - 3s/epoch - 8ms/step
Epoch 2/10
300/300 - 3s - loss: 0.1535 - accuracy: 0.9455 - val_loss: 0.1211 - val_accuracy: 0.9588 - 3s/epoch - 9ms/step
Epoch 3/10
300/300 - 3s - loss: 0.1486 - accuracy: 0.9478 - val_loss: 0.1203 - val_accuracy: 0.9582 - 3s/epoch - 9ms/step
Epoch 4/10
300/300 - 3s - loss: 0.1473 - accuracy: 0.9469 - val_loss: 0.1186 - val_accuracy: 0.9594 - 3s/epoch - 9ms/step
Epoch 5/10
300/300 - 3s - loss: 0.1463 - accuracy: 0.9476 - val_loss: 0.1205 - val_accuracy: 0.9588 - 3s/epoch - 9ms/step
Epoch 6/10
300/300 - 2s - loss: 0.1453 - accuracy: 0.9473 - val_loss: 0.1184 - val_accuracy: 0.9608 - 2s/epoch - 8ms/step
Epoch 7/10
300/300 - 3s - loss: 0.1412 - accuracy: 0.9490 - val_loss: 0.1167 - val_accuracy: 0.9610 - 3s/epoch - 9ms/step
Epoch 8/10
300/300 - 3s - loss: 0.1395 - accuracy: 0.9488 - val_loss: 0.1202 - val_accuracy: 0.9572 - 3s/epoch - 8ms/step
Epoch 9/10
300/300 - 3s 